In [2]:
!pip install azureml.core

  Using cached azureml_core-1.25.0-py3-none-any.whl (2.2 MB)
  Using cached azure_common-1.1.27-py2.py3-none-any.whl (12 kB)
  Using cached pathspec-0.8.1-py2.py3-none-any.whl (28 kB)
  Using cached azure_mgmt_keyvault-2.2.0-py2.py3-none-any.whl (89 kB)
  Using cached azure_mgmt_containerregistry-2.8.0-py2.py3-none-any.whl (718 kB)
  Using cached ndg_httpsclient-0.5.1-py3-none-any.whl (34 kB)
  Using cached jsonpickle-2.0.0-py2.py3-none-any.whl (37 kB)
  Using cached azure_mgmt_storage-11.2.0-py2.py3-none-any.whl (547 kB)
  Using cached azure_graphrbac-0.61.1-py2.py3-none-any.whl (141 kB)
  Using cached msrestazure-0.6.4-py2.py3-none-any.whl (40 kB)
  Using cached azure_mgmt_resource-12.1.0-py2.py3-none-any.whl (1.1 MB)
  Using cached msrest-0.6.21-py2.py3-none-any.whl (85 kB)
  Using cached adal-1.2.6-py2.py3-none-any.whl (55 kB)
  Using cached jmespath-0.10.0-py2.py3-none-any.whl (24 kB)
  Using cached azure_mgmt_authorization-0.61.0-py2.py3-none-any.whl (94 kB)
  Using cached docker

In [36]:
from azureml.core import Workspace
ws = Workspace.create(name='Loan_Predict_Deploy_True',
                      subscription_id = 'fill-with-the-subscription-id-from-azure',
                      resource_group = 'new_loan_data2',
                      create_resource_group = True,
                      location='westeurope')

Deploying StorageAccount with name loanpredstoragea2f46cc3e.
Deploying AppInsights with name loanpredinsights6d8d41f0.
Deployed AppInsights with name loanpredinsights6d8d41f0. Took 9.08 seconds.
Deploying KeyVault with name loanpredkeyvaulte5712573.
Deployed KeyVault with name loanpredkeyvaulte5712573. Took 25.75 seconds.
Deployed StorageAccount with name loanpredstoragea2f46cc3e. Took 30.9 seconds.
Deploying Workspace with name Loan_Predict_Deploy_True.
Deployed Workspace with name Loan_Predict_Deploy_True. Took 42.58 seconds.


In [37]:
from azureml.core.model import Model
model = Model.register(model_path = 'model.onnx',
                       model_name = 'Loan_Predict_Model',
                       description = 'Testing Loan Model Deployment',
                       workspace = ws)

Registering model Loan_Predict_Model


In [38]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "myenv.yml",
                                                  description = "Loan_prediction"
                                                 )

<ipython-input-38-26aa655c1ccd>:3: DeprecationWarning: ContainerImage class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  image_config = ContainerImage.image_configuration(execution_script = "score.py",


In [39]:
image = ContainerImage.create(name = "loanmodel5",
                              models = [model],
                              image_config = image_config,
                              workspace = ws)
image.wait_for_creation(show_output = True)


<ipython-input-39-da58477f56ca>:1: DeprecationWarning: Image class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  image = ContainerImage.create(name = "loanmodel5",


Creating image
Running............................................
Succeeded
Image creation operation finished for image loanmodel5:1, operation "Succeeded"


In [40]:
from azureml.core.webservice import AciWebservice, Webservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {"data": "Loan", "type": "classification"}, 
                                               description = 'Default Loan Prediction')

service_name = 'loan-default-prediction8'
service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                            image = image,
                                            name = service_name,
                                            workspace = ws)

service.wait_for_deployment(show_output = True)
print(service.state)

<ipython-input-40-ab945b9264bf>:9: DeprecationWarning: deploy_from_image has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  service = Webservice.deploy_from_image(deployment_config = aciconfig,


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-03-31 19:49:59+07:00 Generating deployment configuration.
2021-03-31 19:49:59+07:00 Submitting deployment to compute..
2021-03-31 19:50:42+07:00 Checking the status of deployment loan-default-prediction8..
2021-03-31 19:57:13+07:00 Checking the status of inference endpoint loan-default-prediction8.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [41]:
service.scoring_uri

'http://e3269d5d-018b-4ce0-a2d7-32f445789165.westeurope.azurecontainer.io/score'

In [ ]:
#testing
import request